# Building a ReAct agent from scratch using Ollama

In this tutorial, we'll build a simple ReAct (Reasoning and Acting) agent using Ollama The agent implements the Thought -> Action -> Observation (Pause) loop to answer user queries effectively.

By using simple tools, the agent can extend its capabilities beyond mere language processing, enabling it to interact with data sources and perform basic computations as needed.


In [53]:
from openai import OpenAI
import re
import openai

from dotenv import load_dotenv
_ = load_dotenv()

# Optional sanity prints (same idea as your original)
print("OpenAI class:", OpenAI)
print("openai module path:", openai.__file__)

# Ollama uses an OpenAI-compatible API at /v1
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",  # any non-empty string; Ollama ignores it
)



OpenAI class: <class 'openai.OpenAI'>
openai module path: /Users/mohammadmeftauddin/PycharmProjects/Feb16_BuildingAIAgents/.venv/lib/python3.9/site-packages/openai/__init__.py


In [54]:
class Agent:
    """
    A class representing an AI agent that can engage in conversations using OpenAI's API.
    """

    def __init__(self, system=""):
        """
        Initialize the agent with an optional system message.

        Args:
            system (str): The system message to set the context for the agent.
        """
        self.system = system
        self.messages = []
        if self.system:
            # If a system message is provided, add it to the message history
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        """
        Allow the agent to be called directly with a message.

        Args:
            message (str): The user's input message.

        Returns:
            str: The agent's response to the input message.
        """
        # Add the user's message to the conversation history
        self.messages.append({"role": "user", "content": message})
        # Execute the conversation and get the result
        result = self.execute()
        # Add the assistant's response to the conversation history
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        """
        Execute the conversation by sending the entire conversation history to the OpenAI API.

        Returns:
            str: The content of the model's response.
        """
        # Send the entire conversation history to the OpenAI API
        completion = client.chat.completions.create(
                        model="gpt-oss:120b-cloud",
                        temperature=0,
                        messages=self.messages)
        # Return the content of the model's response
        return completion.choices[0].message.content

In [32]:
# Define a prompt for the agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_total_price:
e.g. calculate_total_price: apple: 2, banana: 3
Runs a calculation for the total price based on the quantity and prices of the fruits.

get_fruit_price:
e.g. get_fruit_price: apple
returns the price of the fruit when given its name.

Example session:

Question: What is the total price for 2 apples and 3 bananas?
Thought: I should calculate the total price by getting the price of each fruit and summing them up.
Action: get_fruit_price: apple
PAUSE

Observation: The price of an apple is $1.5.

Action: get_fruit_price: banana
PAUSE

Observation: The price of a banana is $1.2.

Action: calculate_total_price: apple: 2, banana: 3
PAUSE

You then output:

Answer: The total price for 2 apples and 3 bananas is $6.6.
""".strip()

In [55]:
prompt = """
You are a tool-using agent.

You MUST follow this exact format.

If you need to use a tool, output ONLY:
Action: <tool_name>: <tool_input>
PAUSE

After I reply with:
Observation: <result>
you may call another tool (same format) or finish.

When you are ready to respond to the user, output ONLY:
Answer: <final answer>

Rules:
- Do NOT output "Thought:".
- Do NOT include any extra text before/after the formats.
- Do NOT put Action and Answer in the same message.
- If you output Action, you MUST output PAUSE on the next line.

Available actions:
- get_fruit_price: <fruit>
- calculate_total_price: <fruit: qty, fruit: qty>

Example:
Action: get_fruit_price: apple
PAUSE
""".strip()

In [56]:
# Price lookup for fruits
fruit_prices = {
    "apple": 1.5,
    "banana": 1.2,
    "orange": 1.3,
    "grapes": 2.0
}

# Function to calculate the price of a specific fruit
def get_fruit_price(fruit):
    if fruit in fruit_prices:
        return f"The price of a {fruit} is ${fruit_prices[fruit]}"
    else:
        return f"Sorry, I don't know the price of {fruit}."

# Function to calculate total price based on quantities
def calculate_total_price(fruits):
    total = 0.0
    fruit_list = fruits.split(", ")
    for item in fruit_list:
        fruit, quantity = item.split(": ")
        quantity = int(quantity)
        if fruit in fruit_prices:
            total += fruit_prices[fruit] * quantity
        else:
            return f"Sorry, I don't have the price of {fruit}."
    return f"The total price is ${total:.2f}"

# Mapping actions to functions
known_actions = {
    "get_fruit_price": get_fruit_price,
    "calculate_total_price": calculate_total_price
}

In [57]:
react_agent = Agent(system=prompt)

In [58]:
# Run a query
action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to select action

def query(question):
    bot = Agent(prompt)
    result = bot(question)
    print(result)
    actions = [
        action_re.match(a)
        for a in result.split('\n')
        if action_re.match(a)
    ]
    if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
            raise Exception(f"Unknown action: {action}: {action_input}")
        print(f" -- running {action} {action_input}")
        observation = known_actions[action](action_input)
        print("Observation:", observation)
    else:
        return

In [16]:
query("What is the price of a banana?")

Thought: Need price of banana.Action: get_fruit_price: bananaAnswer: The price of a banana is $1.2.


In [41]:
query("What is the price of 2 bananas?")

Thought: I need to find the price of a banana, then calculate the total for 2 bananas.


In [59]:
# Run a query
action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to select action

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return

In [60]:
query("What is the price of 2 bananas?")

Action: calculate_total_price: banana: 2
PAUSEAnswer: The total price for 2 bananas is $2.00.
 -- running calculate_total_price banana: 2
Observation: The total price is $2.40
Answer: The total price for 2 bananas is $2.40.


In [65]:
query("If i bought 10 apples, 10 bananas, and 2 oranges, how much would it cost?")

Action: get_fruit_price: apple
PAUSEAction: get_fruit_price: banana
PAUSEAction: get_fruit_price: orange
PAUSEAnswer: I’m unable to provide the total cost because the fruit price information was not returned. Please provide the prices for apples, bananas, and oranges.
 -- running get_fruit_price apple
Observation: The price of a apple is $1.5
Action: get_fruit_price: banana
PAUSEAction: get_fruit_price: orange
PAUSEAnswer: I’m still missing the prices for bananas and oranges. Please provide those so I can calculate the total cost.
 -- running get_fruit_price banana
Observation: The price of a banana is $1.2
Action: get_fruit_price: orange
PAUSE
 -- running get_fruit_price orange
Observation: The price of a orange is $1.3
Action: calculate_total_price: apple: 10, banana: 10, orange: 2
PAUSE
 -- running calculate_total_price apple: 10, banana: 10, orange: 2
Observation: The total price is $29.60
Answer: The total cost for 10 apples, 10 bananas, and 2 oranges is $29.60.


In [64]:
query("tell me joke about tennis")

Answer: Why did the tennis player bring a ladder to the match? Because they heard the stakes were high!
